In [1]:
%load_ext autoreload
%autoreload 2

env = None

## Import policy and env

In [3]:
from IPython.display import Video

import os
import numpy as np
import json
import pybullet as p
import imageio

from PIL import Image
from nerf_rrc import utils
from nerf_rrc.viz_utils import VisualMarkers
from nerf_rrc.pd_control import PDControlPolicy
from nerf_rrc.cube_trajectory_env import ActionType, SimCubeTrajectoryEnv
from nerf_grasping.control import pos_control

# comment if running remote with VNC
os.environ["DISPLAY"] = ":1"

## Helper funcs

In [5]:
import os
import pybullet as p
import glob
from PIL import Image


def get_vid_path():
    i = 0
    vid_path = lambda: f'vids/run{i}.mp4'
    if not os.path.exists(os.path.split(vid_path())[0]):
        os.makedirs(os.path.split(vid_path())[0])
    while os.path.exists(vid_path()):
        i += 1
    return vid_path()


def write_frame_to_image(img_path, **camera_kwargs):
    frame = p.getCameraImage(**camera_kwargs)[2]
    im = Image.fromarray(frame)
    im.save(img_path)
    return


def write_frames_to_video(frames=None, img_dir="", fps=20):
    if img_dir:
        frames = sorted(glob.glob(os.path.join(img_dir, "img*.png")),
                        key=lambda x: int(x.split("img")[1].split(".")[0]))
    vid_path = get_vid_path()
    writer = imageio.get_writer(vid_path, fps=20)  # actual fps is 1000 // save_freq ~~ 200
    for image in frames:
        if img_dir:
            image = imageio.imread(file)
        writer.append_data(image)
    writer.close()
    print(f"wrote frames to {vid_path}")
    return vid_path


def debug_markers(markers, grasp_pts, marker_colors=None):
    # default to RGB
    if marker_colors is None:
        marker_colors = [(1., 0., 0., .5), 
                         (0., 1., 0., .5), 
                         (0., 0., 1., .5)]
    if markers is None:
        markers = VisualMarkers()
        for grasp_pt, color in zip(grasp_pts, marker_colors):
            markers.add(grasp_pt, color=color)
    else:
        for marker, grasp_pt in zip(markers.markers, grasp_pts):
            marker.set_state(grasp_pt)
    return markers


def load_debug_camera_viz_configs(config_path='./nerf_rrc/debug_camera.npz'):
    """Loads debug camera and visualizer kwargs"""
    debug_camera_kwargs = dict(np.load(config_path))
    visualizer_kws = ['cameraDistance', 'cameraYaw', 'cameraPitch', 'cameraTargetPosition']
    camera_kws = ['width', 'height', 'viewMatrix', 'projectionMatrix']
    scalar_kws = ['width', 'height', 'cameraYaw', 'cameraPitch', 'cameraDistance']
    for k in scalar_kws:
        debug_camera_kwargs[k] = debug_camera_kwargs[k].item()

    debug_camera_kwargs['width'] = debug_camera_kwargs['width'] // 4
    debug_camera_kwargs['height'] = debug_camera_kwargs['height'] // 4

    camera_kwargs = {k: debug_camera_kwargs[k] for k in camera_kws}
    visualizer_kwargs = {k: debug_camera_kwargs[k] for k in visualizer_kws}
    return camera_kwargs, visualizer_kwargs


def calc_gravity(observation, inv_dyn=False):
    q0 = observation['robot_observation']['position']
    dq0 = observation['robot_observation']['velocity']
    if inv_dyn:
        return np.asarray(p.calculateInverseDynamics(
            env.platform.simfinger.finger_id,
            list(q0), list(dq0),
            [0.0 for _ in range(9)]
        ))

    jac_g = 0
    for fid in range(3):
        frame_ids = [
                policy.kinematics.finger_link_ids[f'finger_{link}_link_{fid * 120}'] + 2
                    for link in ['upper', 'middle', 'lower']]
        frame_names = [policy.kinematics.robot_model.frames[i].name for i in frame_ids]
        print(frame_names)
        ms = policy.kinematics.ms
        for i, frame_id in enumerate(frame_ids):
            grav = np.array([0, 0, -9.8 * ms[i]])
            jac_g -= policy.kinematics.compute_jacobian(frame_id, q0)[:3,:].T @ grav

    return jac_g

## Run episode boilerplate

Close and delete env and policy
```python
env.close()
del env
del policy
```

Camera visualization params

```python
(width,
height,
viewMatrix,
projectionMatrix,
cameraUp,
cameraForward,
horizontal,
vertical,
cameraYaw,
cameraPitch,
cameraDistance,
target) = p.getDebugVisualizerCamera()

np.savez('debug_camera.npz', width=width, height=height,
         viewMatrix=viewMatrix, projectionMatrix=projectionMatrix,
         cameraYaw=cameraYaw, cameraPitch=cameraPitch, 
         cameraDistance=cameraDistance, cameraTargetPosition=target)
```

Load pinocchio utils from benchmark-rrc

```python
from rrc.env import pinocchio_utils
import os.path as osp
import trifinger_simulation

robot_properties_path = osp.join(osp.split(osp.abspath(trifinger_simulation.__file__))[0],
                                         "robot_properties_fingers")
urdf_file = trifinger_simulation.finger_types_data.get_finger_urdf(
    "trifingerpro"
)
finger_urdf_path = osp.join(robot_properties_path, "urdf", urdf_file)
pi = pinocchio_utils.PinocchioUtils(finger_urdf_path)
pi.compute_jacobian = pi.get_any_link_jacobian
pi.compute_lambda_and_g_matrix = pi.get_lambda_and_g_matrix
```

## Initialize policy and env

In [6]:
# initialize visualization vars
save_vid = False
save_freq = 10
visualization = True

camera_kwargs, visualizer_kwargs = load_debug_camera_viz_configs()

# load goal
goal_json = './goal.json'
goal = json.loads(open(goal_json, "r").read())["_goal"]

# create env
if env is None:
    env = SimCubeTrajectoryEnv(
        goal_trajectory=goal,  # passing None to sample a random trajectory
        action_type=ActionType.TORQUE,
        visualization=visualization,
    )

# initialize loop vars and visualizer (if visualization==True)
is_done = False
observation = env.reset()
p.resetDebugVisualizerCamera(**visualizer_kwargs)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)

t = 0

# create policy
policy = PDControlPolicy(env.action_space, env.info["trajectory"])

## Computing gravity compensation forces

```python
import time
q0 = observation['robot_observation']['position']
dq0 = observation['robot_observation']['velocity']
g2 = np.asarray(p.calculateInverseDynamics(
    env.platform.simfinger.finger_id,
    list(q0), list(dq0),
    [0.0 for _ in range(9)]
))

g_tot = 0
for fid in [0,120,240]:
    frame_ids = [policy.kinematics.finger_link_ids[f'finger_{link}_link_{fid}'] + 1 for link in ['upper', 'middle', 'lower']]
    ms = policy.kinematics.ms
    for i, frame_id in enumerate(frame_ids):
        grav = np.array([0, 0, -9.8 * ms[i]]) 
        g_tot -= policy.kinematics.compute_jacobian(frame_id, q0)[:3,:].T @ grav
```

In [5]:
calc_gravity(observation)

['finger_middle_link_0', 'finger_lower_link_0', 'finger_tip_link_0']


AttributeError: 'Kinematics' object has no attribute 'compute_jacobian'

In [ ]:
# first write after reset
is_done = False
observation = env.reset()
p.resetDebugVisualizerCamera(**visualizer_kwargs)
t = 0

markers = None
last_mode = ""

q0 = observation['robot_observation']['position']
dq0 = observation['robot_observation']['velocity']
inv_dyn_g = np.asarray(p.calculateInverseDynamics(
    env.platform.simfinger.finger_id,
    list(q0), list(dq0),
    [0.0 for _ in range(9)]
))


while not is_done:
    action = policy.predict(observation, t=10)  # always on off mode
    gravity_comp = calc_gravity(observation, inv_dyn=False)
    observation, reward, is_done, info = env.step(gravity_comp)
    t = info["time_index"]
    grasp_pts, normals = policy.compute_grasp_points_normals(observation, overwrite_z=True)
    if policy.set_mode(t) != last_mode:
        last_mode = policy.set_mode(t)
        if (~np.isclose(grasp_pts[:, 2], utils.CUBE_HALF_SIZE, atol=1e-6)).any():
            print(grasp_pts[:, 2] - utils.CUBE_HALF_SIZE)
            pdb.set_trace()
        markers = debug_markers(markers, grasp_pts)
        for m in markers.markers:
            print(m.position)
    if save_vid and t % save_freq == 0:
        write_frame_to_image(f"/tmp/img{t}.png", **camera_kwargs)

if save_vid:
    write_frames_to_video(img_dir="/tmp")

## Moving fingertips up and down

In [25]:
control_params = pos_control.load_config("pos_control.yaml")

In [26]:
control_params

PosControlConfig(Kp=20.0, Kd=[0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5], damping=1e-12)

In [27]:
# first write after reset
is_done = False
observation = env.reset()
p.resetDebugVisualizerCamera(**visualizer_kwargs)
t = 0
control_params = pos_control.load_config("pos_control.yaml")
kin = policy.kinematics
orig_tp =  np.asarray(
        policy.kinematics.forward_kinematics(
            observation["robot_observation"]["position"]
        )
    ).flatten()

while t < 5000:
    min_height, max_height = 0.05, 0.1
    h = .5 * (1 + np.sin(np.pi / 500 * t)) * (max_height - min_height) + min_height
    tip_pos = orig_tp.copy()
    tip_pos[2::3] = h
    q, dq = observation['robot_observation']['position'], observation['robot_observation']['velocity']
    action = pos_control.get_joint_torques(tip_pos, kin.robot_model, kin.data,
                                            q, dq, control_params)
    # action = policy.position_pd_control(observation, tip_pos)
    # action += policy.gravity_comp(observation)
    action = policy.clip_to_space(action)
    observation, reward, is_done, info = env.step(action)
    t  = info['time_index']

rofile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded usin

## Running episode with PDControl Policy

In [7]:
# first write after reset
if save_vid:
    write_frame_to_image(f"/tmp/img{t}.png", **camera_kwargs)

markers = None
last_mode = ""

while not is_done:
    action = policy.predict(observation, t)  # always on off mode
    observation, reward, is_done, info = env.step(action)
    t = info["time_index"]
    grasp_pts, normals = policy.sample_grasp(observation, overwrite_z=True)
    if policy.set_mode(t) != last_mode:
        last_mode = policy.set_mode(t)
        if (~np.isclose(grasp_pts[:, 2], utils.CUBE_HALF_SIZE, atol=1e-6)).any():
            print(grasp_pts[:, 2] - utils.CUBE_HALF_SIZE)
            pdb.set_trace()
        markers = debug_markers(markers, grasp_pts)
    if save_vid and t % save_freq == 0:
        write_frame_to_image(f"/tmp/img{t}.png", **camera_kwargs)

if save_vid:
    write_frames_to_video(img_dir="/tmp")

first choice face: 2
first choice face: 5
first choice face: 1
[2. 1. 5.]
assigning cp params for lifting
[array([0., 1., 0.]), array([ 0., -1.,  0.]), array([-1.,  0.,  0.])]
first choice face: 2
first choice face: 5
first choice face: 1
[2. 1. 5.]
assigning cp params for lifting
[array([0., 1., 0.]), array([ 0., -1.,  0.]), array([-1.,  0.,  0.])]
first choice face: 2
first choice face: 5
first choice face: 1
[2. 1. 5.]
assigning cp params for lifting
[array([0., 1., 0.]), array([ 0., -1.,  0.]), array([-1.,  0.,  0.])]
first choice face: 2
first choice face: 5
first choice face: 1
[2. 1. 5.]
assigning cp params for lifting
[array([0., 1., 0.]), array([ 0., -1.,  0.]), array([-1.,  0.,  0.])]
first choice face: 2
first choice face: 5
first choice face: 1
[2. 1. 5.]
assigning cp params for lifting
[array([0., 1., 0.]), array([ 0., -1.,  0.]), array([-1.,  0.,  0.])]
first choice face: 2
first choice face: 5
first choice face: 1
[2. 1. 5.]
assigning cp params for lifting
[array([0., 1.

AssertionError: END OF EPISODE

In [48]:
grasp_pts[:, 2], utils.CUBE_HALF_SIZE

(array([0.0325, 0.0325, 0.0325]), 0.0335)

In [15]:
[p.getBodyInfo(x.body_id) for x in markers.markers]

[(b'link0', b''), (b'link0', b''), (b'link0', b'')]

In [38]:
import matplotlib.pyplot as plt

yzs = np.concatenate([gp[:, 1:] for gp in grasp_pt_hist])
xzs = np.concatenate([gp[:, [0,2]] for gp in grasp_pt_hist])

f, ax = plt.subplots(ncols=2, figsize=(12,6))
ax[0].scatter(yzs[:, 0], yzs[:, 1])
ax[1].scatter(xzs[:, 0], xzs[:, 1])

ValueError: need at least one array to concatenate

In [59]:
action

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [43]:
if save_vid:
    write_frames_to_video(frames)

wrote frames to vids/run0.mp4


In [44]:

Video("vids/run0.mp4")